# Data Cleaning

This is the data cleaning notebook,

Import libraries

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import re
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, mean_squared_error, accuracy_score, precision_score, recall_score

### Data dictionary

- **PassengerId** - A unique Id for each passenger. Each Id takes the form ```gggg_pp``` where ```gggg``` indicates a group the passenger is travelling with and ```pp``` is their number within the group. People in a group are often family members, but not always.
- **HomePlanet** - The planet the passenger departed from, typically their planet of permanent residence.
- **CryoSleep** - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.
- **Cabin** - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.
- **Destination** - The planet the passenger will be debarking to.
- **Age** - The age of the passenger.
- **VIP** - Whether the passenger has paid for special VIP service during the voyage.
- **RoomService, FoodCourt, ShoppingMall, Spa, VRDeck** - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.
- **Name** - The first and last names of the passenger.
- **Transported** - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.

let's load the data

In [2]:
df = pd.read_csv('../data/raw/train.csv')
# df = pd.read_csv('../data/train.csv', dtype_backend='pyarrow')

## Data Cleaning

Drop unique columns

In [3]:
df = df.drop(['Name'], axis=1)
print(df.shape)

(8693, 13)


Check for missing values 

In [4]:
df.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Transported       0
dtype: int64

Filling the gaps....

HomePlanet

In [5]:
# print('Printing most repeated entry: ')
# print(df['HomePlanet'].value_counts().index[0])
print('Printing the mode')
print(df['HomePlanet'].mode())

Printing the mode
0    Earth
Name: HomePlanet, dtype: object


In [6]:
df['HomePlanet'].isnull().sum()

201

In [7]:
df['HomePlanet'].fillna(df['HomePlanet'].mode()[0], inplace=True)

In [8]:
def fillmode(df_series: str) -> None:
    """ Fill the series's null values with its mode

    Args:
        df_series (str): series to fill
    """
    df[df_series].fillna(df[df_series].mode()[0], inplace=True)

In [9]:
def fillzeros (df_series: str) -> None:
    """ Fill the series's null values with 0 (float)

    Args:
        df_series (str): series to fill
    """
    df[df_series].fillna(0, inplace=True)

CryoSleep

In [10]:
fillmode('CryoSleep')

Cabin

In [11]:
# fillzeros('Cabin')
# Used this in the beginning but a single 0 isn't compliant with df['Cabin']'s pattern

In [12]:
df['Cabin'].fillna('0/0/0', inplace=True)

Destination

In [13]:
fillmode('Destination')

Age

In [14]:
df['Age'].median()

27.0

In [15]:
def fillmedian(df_series: str) -> None:
    """ Fill the series's null values with the feature's median

    Args:
        df_series (str): series to fill
    """
    df[df_series].fillna(df[df_series].median(), inplace=True)

In [16]:
fillmedian('Age')

VIP

In [17]:
fillmode('VIP')

Amenities

In [18]:
fillzeros('FoodCourt')
fillzeros('ShoppingMall')
fillzeros('Spa')
fillzeros('VRDeck')
fillzeros('RoomService')

Checking again

In [19]:
df.isna().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64

In [20]:
df.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000
mean,28.790291,220.009318,448.434027,169.572300,304.588865,298.261820
std,14.341404,660.519050,1595.790627,598.007164,1125.562559,1134.126417
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,37.000000,41.000000,61.000000,22.000000,53.000000,40.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [21]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


I can archieve the same done in this notebook with the SimpleImputer() class of sklearn

In [22]:
# Reset the dataframe
raw_df = pd.read_csv('../data/raw/train.csv')
# df = pd.read_csv('../data/train.csv', dtype_backend='pyarrow')

In [23]:
raw_df.drop('Name', axis=1, inplace=True)

In [24]:
s_imputer_median = SimpleImputer(strategy='median')
s_imputer_mode = SimpleImputer(strategy='most_frequent')
s_imputer_const = SimpleImputer(strategy='constant', fill_value=0)
s_imputer_const_cab = SimpleImputer(strategy='constant', fill_value='0/0/0')
median_features = ['Age']
mode_features = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']
const_features = ['FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'RoomService']
const_cab_features = ['Cabin']

In [25]:
# The ColumnTransformer class needs a list of transformers, these transformers are tuples of 3 values: 
# Name of the step, the transformer to run, the objects affected by the step
# The remainder parameter controls what to do with the columns not involved in the ColumnTransformer
# Remainder default value = 'drop', drop the others column in the output 
# The columns in the output are ordered by their step, first in first out 
# The verbose parameter makes the ColumnTransformer return the time required to complete their operations
# The verbose_feature_names_out parameter adds a prefix to each column with the stepname that generated it
imputer_transformer = ColumnTransformer([
    ('Median', s_imputer_median, median_features), 
    ('Mode', s_imputer_mode, mode_features), 
    ('Const', s_imputer_const, const_features), 
    ('Cabin', s_imputer_const_cab, const_cab_features)],
    remainder='passthrough',
    verbose_feature_names_out=False
    )

In [26]:
imputer_transformer.fit_transform(raw_df)

array([[39.0, 'Europa', False, ..., 'B/0/P', '0001_01', False],
       [24.0, 'Earth', False, ..., 'F/0/S', '0002_01', True],
       [58.0, 'Europa', False, ..., 'A/0/S', '0003_01', False],
       ...,
       [26.0, 'Earth', False, ..., 'G/1500/S', '9279_01', True],
       [32.0, 'Europa', False, ..., 'E/608/S', '9280_01', False],
       [44.0, 'Europa', False, ..., 'E/608/S', '9280_02', True]],
      dtype=object)

In [27]:
imputer_columns = imputer_transformer.get_feature_names_out()
print(imputer_columns)

['Age' 'HomePlanet' 'CryoSleep' 'Destination' 'VIP' 'FoodCourt'
 'ShoppingMall' 'Spa' 'VRDeck' 'RoomService' 'Cabin' 'PassengerId'
 'Transported']


In [28]:
imputer_df = pd.DataFrame(imputer_transformer.fit_transform(df), columns=imputer_columns)
imputer_df.head()

,Age,HomePlanet,CryoSleep,Destination,VIP,FoodCourt,ShoppingMall,Spa,VRDeck,RoomService,Cabin,PassengerId,Transported
0,39.0,Europa,False,TRAPPIST-1e,False,0.0,0.0,0.0,0.0,0.0,B/0/P,0001_01,False
1,24.0,Earth,False,TRAPPIST-1e,False,9.0,25.0,549.0,44.0,109.0,F/0/S,0002_01,True
2,58.0,Europa,False,TRAPPIST-1e,True,3576.0,0.0,6715.0,49.0,43.0,A/0/S,0003_01,False
3,33.0,Europa,False,TRAPPIST-1e,False,1283.0,371.0,3329.0,193.0,0.0,A/0/S,0003_02,False
4,16.0,Earth,False,TRAPPIST-1e,False,70.0,151.0,565.0,2.0,303.0,F/1/S,0004_01,True


In [29]:
imputer_df.describe()

,Age,HomePlanet,CryoSleep,Destination,VIP,FoodCourt,ShoppingMall,Spa,VRDeck,RoomService,Cabin,PassengerId,Transported
count,8693.0,8693,8693,8693,8693,8693.0,8693.0,8693.0,8693.0,8693.0,8693,8693,8693
unique,80.0,3,2,3,2,1507.0,1115.0,1327.0,1306.0,1273.0,6561,8693,2
top,27.0,Earth,False,TRAPPIST-1e,False,0.0,0.0,0.0,0.0,0.0,0/0/0,0001_01,True
freq,438.0,4803,5656,6097,8494,5639.0,5795.0,5507.0,5683.0,5758.0,199,1,4378


The output is different because all the numeric features in the imputer_df are objects instead of floats

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8693 non-null   object 
 2   CryoSleep     8693 non-null   bool   
 3   Cabin         8693 non-null   object 
 4   Destination   8693 non-null   object 
 5   Age           8693 non-null   float64
 6   VIP           8693 non-null   bool   
 7   RoomService   8693 non-null   float64
 8   FoodCourt     8693 non-null   float64
 9   ShoppingMall  8693 non-null   float64
 10  Spa           8693 non-null   float64
 11  VRDeck        8693 non-null   float64
 12  Transported   8693 non-null   bool   
dtypes: bool(3), float64(6), object(4)
memory usage: 704.7+ KB


In [31]:
imputer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Age           8693 non-null   object
 1   HomePlanet    8693 non-null   object
 2   CryoSleep     8693 non-null   object
 3   Destination   8693 non-null   object
 4   VIP           8693 non-null   object
 5   FoodCourt     8693 non-null   object
 6   ShoppingMall  8693 non-null   object
 7   Spa           8693 non-null   object
 8   VRDeck        8693 non-null   object
 9   RoomService   8693 non-null   object
 10  Cabin         8693 non-null   object
 11  PassengerId   8693 non-null   object
 12  Transported   8693 non-null   object
dtypes: object(13)
memory usage: 883.0+ KB


In [32]:
float_features = ['Age', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'RoomService']

In [33]:
for cols in float_features: 
    imputer_df[cols] = imputer_df[cols].astype('float')

In [34]:
imputer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Age           8693 non-null   float64
 1   HomePlanet    8693 non-null   object 
 2   CryoSleep     8693 non-null   object 
 3   Destination   8693 non-null   object 
 4   VIP           8693 non-null   object 
 5   FoodCourt     8693 non-null   float64
 6   ShoppingMall  8693 non-null   float64
 7   Spa           8693 non-null   float64
 8   VRDeck        8693 non-null   float64
 9   RoomService   8693 non-null   float64
 10  Cabin         8693 non-null   object 
 11  PassengerId   8693 non-null   object 
 12  Transported   8693 non-null   object 
dtypes: float64(6), object(7)
memory usage: 883.0+ KB


In [35]:
imputer_df.describe()

,Age,FoodCourt,ShoppingMall,Spa,VRDeck,RoomService
count,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000
mean,28.790291,448.434027,169.572300,304.588865,298.261820,220.009318
std,14.341404,1595.790627,598.007164,1125.562559,1134.126417,660.519050
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,37.000000,61.000000,22.000000,53.000000,40.000000,41.000000
max,79.000000,29813.000000,23492.000000,22408.000000,24133.000000,14327.000000


In [36]:
imputer_df.isnull().sum()

Age             0
HomePlanet      0
CryoSleep       0
Destination     0
VIP             0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
RoomService     0
Cabin           0
PassengerId     0
Transported     0
dtype: int64

The same output as before 

### Saving the CSV

In [37]:
df.to_csv('../data/stg/train_stg.csv', index=False)